In [118]:
import pandas as pd

# Loading the data file
data = pd.read_csv("diabetic_data.csv")
print(data)

        encounter_id  patient_nbr             race  gender      age weight  \
0            2278392      8222157        Caucasian  Female   [0-10)      ?   
1             149190     55629189        Caucasian  Female  [10-20)      ?   
2              64410     86047875  AfricanAmerican  Female  [20-30)      ?   
3             500364     82442376        Caucasian    Male  [30-40)      ?   
4              16680     42519267        Caucasian    Male  [40-50)      ?   
...              ...          ...              ...     ...      ...    ...   
101761     443847548    100162476  AfricanAmerican    Male  [70-80)      ?   
101762     443847782     74694222  AfricanAmerican  Female  [80-90)      ?   
101763     443854148     41088789        Caucasian    Male  [70-80)      ?   
101764     443857166     31693671        Caucasian  Female  [80-90)      ?   
101765     443867222    175429310        Caucasian    Male  [70-80)      ?   

        admission_type_id  discharge_disposition_id  admission_

In [119]:
# Using the columns [class, age, time_in_hospital] we can melt the following together
melted_data = data.melt(id_vars=['patient_nbr'], value_vars=['age', 'time_in_hospital'],
                      var_name='Health_Metric', value_name='Value')
print(melted_data)
# This matches the patient number to other values (the age of the patient and the time spent inside of the hospital

        patient_nbr     Health_Metric    Value
0           8222157               age   [0-10)
1          55629189               age  [10-20)
2          86047875               age  [20-30)
3          82442376               age  [30-40)
4          42519267               age  [40-50)
...             ...               ...      ...
203527    100162476  time_in_hospital        3
203528     74694222  time_in_hospital        5
203529     41088789  time_in_hospital        1
203530     31693671  time_in_hospital       10
203531    175429310  time_in_hospital        6

[203532 rows x 3 columns]


In [64]:
# Aggregation
average_hospital_time = data.agg({'time_in_hospital': 'mean'})
print(average_hospital_time)

# The average hospital time is what I believe the only useful piece of information to aggregaten without using a groupby function. It wouldn't make 
# sense to aggregate something like Patient ID number, as they're only useful in their original form. It would be nice to aggregate the age, but 
# it's not helpful that each age is just a range of ages stored as a string.

time_in_hospital    4.395987
dtype: float64


In [96]:
# Iterating over the rows of data to print cohesive statements

# for index, row in data.iterrows():
#     print(f'Patient number {row['patient_nbr']} is {row['race']} and takes {row['insulin']} insulin')

# I'm leaving these commented out because they print a large amount of data.


In [97]:
# We can do a lot to change which patients are printed

# for index, row in data.iterrows():
#     if (row['patient_nbr'] % 2 == 0):
#         print(f'Patient number {row['patient_nbr']} is {row['race']} and takes {row['insulin']} insulin')

# This only prints even patient numbers

In [98]:
# We can also only get patients that take a certain type of insulin

# for index, row in data.iterrows():
#     if (row['insulin'] == "No"):
#         print(f'Patient number {row['patient_nbr']} is {row['race']} and takes {row['insulin']} insulin')

In [99]:
# We can also ensure that we only get patients with no data missing

# for index, row in data.iterrows():
#     if ((row['insulin'] != "?") & (row['race'] != "?")):
#         print(f'Patient number {row['patient_nbr']} is {row['race']} and takes {row['insulin']} insulin')

In [ ]:
# With groupby, there are many many things we can do

In [88]:
# One useful thing to do is see how many of each race we have
print(data.groupby(['race']).size())

race
?                   2273
AfricanAmerican    19210
Asian                641
Caucasian          76099
Hispanic            2037
Other               1506
dtype: int64


In [87]:
# We can expand upon this by adding gender to the mix
print(data.groupby(['race', 'gender']).size())

race             gender         
?                Female              1133
                 Male                1138
                 Unknown/Invalid        2
AfricanAmerican  Female             11728
                 Male                7482
Asian            Female               318
                 Male                 323
Caucasian        Female             39689
                 Male               36410
Hispanic         Female              1092
                 Male                 945
Other            Female               748
                 Male                 757
                 Unknown/Invalid        1
dtype: int64


In [95]:
# We can do the same aggregation performed earlier with groupy like so, but with the age
print(data.groupby('age')['time_in_hospital'].mean())

age
[0-10)      2.546584
[10-20)     3.191027
[20-30)     3.564876
[30-40)     3.799735
[40-50)     4.039649
[50-60)     4.125753
[60-70)     4.382244
[70-80)     4.590878
[80-90)     4.808629
[90-100)    4.755818
Name: time_in_hospital, dtype: float64


In [101]:
# The above function is the same as this
print(data.groupby('age').agg({'time_in_hospital': 'mean'}))

          time_in_hospital
age                       
[0-10)            2.546584
[10-20)           3.191027
[20-30)           3.564876
[30-40)           3.799735
[40-50)           4.039649
[50-60)           4.125753
[60-70)           4.382244
[70-80)           4.590878
[80-90)           4.808629
[90-100)          4.755818


In [92]:
# We can group the admission type to the age of the admitted individual, and then aggregate with the value_counts function that simply counts the number of occurences
print(data.groupby('admission_type_id').agg({'age': 'value_counts'}))

                              age
admission_type_id age            
1                 [70-80)   13474
                  [60-70)   11148
                  [80-90)    9878
                  [50-60)    8907
                  [40-50)    5259
...                           ...
8                 [40-50)      26
                  [30-40)      12
                  [20-30)       4
                  [90-100)      4
                  [10-20)       1

[71 rows x 1 columns]


In [114]:
# I ran into many issues with pivot (it doesn't allow duplicates, and this data has a TON of duplicates), but it turns out that a pivot table
# is really good with working with duplicates and is fairly straightforward.

df_filtered = data[data['race'].isin(['?', 'Caucasian'])]  # Filter rows where race is '?' or 'caucasian'

# Create a pivot table to count the occurrences of 'gender' by 'race'
pivot_table = df_filtered.pivot_table(
    index='race', 
    columns='gender', 
    aggfunc='size',  # 'size' will count occurrences
    fill_value = 0 # fills in nan with 0
)

print(pivot_table)

gender     Female   Male  Unknown/Invalid
race                                     
?            1133   1138                2
Caucasian   39689  36410                0


In [117]:
# If we want to use pivot, we can aggregate the data to remove duplicates, and then pivot the aggregated data.
# I had to do a lot of searching to realize as_index needed to be false, and it's because we don't want the columns to become the indices.
df_agg = data.groupby(['race', 'age'], as_index=False).agg({'admission_type_id': 'count'})

df_pivot = df_agg.pivot(index='race', columns='age', values='admission_type_id')

print(df_pivot)

age              [0-10)  [10-20)  [20-30)  [30-40)  [40-50)  [50-60)  [60-70)  \
race                                                                            
?                     1        9       46       76      220      361      495   
AfricanAmerican      16      209      540     1223     2787     4225     4232   
Asian                 2        2        6       15       52      123      160   
Caucasian           136      438      975     2203     6131    11835    16693   
Hispanic              2       23       63      174      337      403      489   
Other                 4       10       27       84      158      309      414   

age              [70-80)  [80-90)  [90-100)  
race                                         
?                    599      397        69  
AfricanAmerican     3816     1817       345  
Asian                180       90        11  
Caucasian          20761    14602      2325  
Hispanic             386      141        19  
Other                326     